file function: generate low level control from high level actions

In [102]:
import argparse

import numpy as np
import pyglet
from gym_duckietown.envs import DuckietownEnv
from pyglet.window import key
import sys
import cv2

In [103]:
# declare the arguments
# parser = argparse.ArgumentParser()

# # You should set them to different map name and seed accordingly
# parser.add_argument('--map-name', '-m', default="map4_0", type=str)
# parser.add_argument('--seed', '-s', default=2, type=int)
# parser.add_argument('--start-tile', '-st', default="1,13", type=str, help="two numbers separated by a comma")
# parser.add_argument('--goal-tile', '-gt', default="3,3", type=str, help="two numbers separated by a comma")
# parser.add_argument('--control_path', default='./map4_0_seed2_start_1,13_goal_3,3.txt', type=str,
#                     help="the control file to run")
# # parser.add_argument('--manual', default=False, type=str2bool, help="whether to manually control the robot")
# args = parser.parse_args()

class Args_:
    def __init__(self):
        # self.map_name = "map4_0"
        # self.seed = 2
        # self.start_tile = "1,13"
        # self.goal_tile = "3,3"
        # self.control_path = 'map4_0_seed2_start_1,13_goal_3,3.txt'

        # add intention
        self.map_name = "map4_0"
        self.seed = 4
        self.start_tile = "3,3"
        self.goal_tile = "10,4"
        self.control_path = 'map4_0_seed4_start_3,3_goal_10,4.txt'

args_ = Args_()
# simulator instantiation
env = DuckietownEnv(
    domain_rand=False,
    max_steps=1500,
    map_name=args_.map_name,
    seed=args_.seed,
    user_tile_start=args_.start_tile,
    goal_tile=args_.goal_tile,
    randomize_maps_on_reset=False
)

print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.shape,
      env.action_space.dtype)

State shape:  (480, 640, 3)
Number of actions:  (2,) float32


> # **Part - I**

*   define actor critic networks
*   define PPO algorithm

notes: Part-I has no modification to PPO_colab.ipynb[https://colab.research.google.com/github/nikhilbarhate99/PPO-PyTorch/blob/master/PPO_colab.ipynb]

In [104]:


############################### Import libraries ###############################


import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np

import gym
# import roboschool
import pybullet_envs


################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()):
    device = torch.device('cuda:0')
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")

print("============================================================================================")




################################## PPO Policy ##################################


class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []
        self.is_terminals = []


    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space, action_std_init):
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_dim = action_dim
            self.action_var = torch.full((action_dim,), action_std_init * action_std_init).to(device)

        # actor
        if has_continuous_action_space :
            self.actor_cnn = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.AdaptiveMaxPool2d(output_size=(1, 1))
            )
            self.actor_fc = nn.Sequential(
                            nn.Linear(4+3, action_dim),
                            nn.Tanh()
                        )
        else:
            self.actor = nn.Sequential(
                            nn.Linear(32, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )

        # critic
        self.critic_cnn = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.AdaptiveMaxPool2d(output_size=(1, 1))
            )

        self.critic_fc = nn.Sequential(
                        nn.Linear(4+3, 16),  ## 此处fc输入特征数值根据报错来修改state_dim
                        nn.Tanh(),
                        nn.Linear(16, 1)
                    )

    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            self.action_var = torch.full((self.action_dim,), new_action_std * new_action_std).to(device)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def forward(self):
        raise NotImplementedError


    def act(self, state, intention):

        onehot_intention = torch.zeros(3, device=intention.device)
        onehot_intention[intention] = 1

        if self.has_continuous_action_space:
            state = torch.tensor(state, dtype=torch.float32).permute(2, 0, 1)
            img_feature = self.actor_cnn(state).squeeze()
            # Cast intention to one-hot encoding
            # intention = intention.unsqueeze(0)

            # Predict control
            action_mean = self.actor_fc(torch.cat((img_feature, onehot_intention), dim=0)).view(-1, self.action_dim)

            cov_mat = torch.diag(self.action_var).unsqueeze(dim=0)
            dist = MultivariateNormal(action_mean, cov_mat)
        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        img_feature = self.critic_cnn(state).squeeze()
        state_val = self.critic_fc(torch.cat((img_feature, onehot_intention), dim=0))
        # state_val = self.critic(state)

        return action.detach(), action_logprob.detach(), state_val.detach()


    def evaluate(self, state, action):

        if self.has_continuous_action_space:
            img_feature = self.actor_cnn(state).flatten(1)
            action_mean = self.actor_fc(img_feature)
            action_var = self.action_var.expand_as(action_mean)
            cov_mat = torch.diag_embed(action_var).to(device)
            dist = MultivariateNormal(action_mean, cov_mat)

            # for single action continuous environments
            if self.action_dim == 1:
                action = action.reshape(-1, self.action_dim)

        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        # state_values = self.critic(state)
        img_feature = self.critic_cnn(state).squeeze()
        state_values = self.critic_fc(img_feature)

        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std_init=0.6):

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor_cnn.parameters(), 'lr': lr_actor},
                        {'params': self.policy.actor_fc.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic_cnn.parameters(), 'lr': lr_critic},
                        {'params': self.policy.critic_fc.parameters(), 'lr': lr_critic},
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()


    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)

        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def decay_action_std(self, action_std_decay_rate, min_action_std):
        # print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                # print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                # print("setting actor output action_std to : ", self.action_std)
                pass

            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        # print("--------------------------------------------------------------------------------------------")


    def select_action(self, state, intention):

        if self.has_continuous_action_space:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state, intention)

            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)
            return action.detach().cpu().numpy().flatten()

        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state)

            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)

        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        old_state_values = torch.squeeze(torch.stack(self.buffer.state_values, dim=0)).detach().to(device)

        # calculate advantages
        advantages = rewards.detach() - old_state_values.detach()


        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)

            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5 * self.MseLoss(state_values, rewards) - 0.01 * dist_entropy

            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()


    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)


    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))




################################# End of Part I ################################


Device set to : cpu


> # **Part - II**

*   train PPO algorithm on environments
*   save preTrained networks weights and log files

In [105]:

print("============================================================================================")
import torch

################################### Training ###################################


####### initialize environment hyper-parameters ######

env_name = "DuckietownEnv"
has_continuous_action_space = True
max_ep_len = 1500           # max timesteps in one episode
action_std = 0.1            # set same std for action distribution which was used while saving
min_action_std = 0
action_std_decay_rate = 0
action_std_decay_freq = 10
####

max_training_timesteps = int(5e3)   # break training loop if timeteps > max_training_timesteps

# print_freq = max_ep_len * 4     # print avg reward in the interval (in num timesteps)
print_freq = 4     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = 100      # save model frequency (in num timesteps)
# save_model_freq = int(2e4)      # save model frequency (in num timesteps)


#####################################################


## Note : print/log frequencies should be > than max_ep_len


################ PPO hyper-parameters ################


update_timestep = max_ep_len * 4      # update policy every n timesteps
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = args_.seed         # set random seed if required (0 = no random seed)

#####################################################



print("training environment name : " + env_name)


# state space dimension
state_dim = env.observation_space.shape[0] * env.observation_space.shape[1]

# action space dimension
if has_continuous_action_space:
    action_dim = env.action_space.shape[0]
else:
    action_dim = env.action_space.n



###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################


################### checkpointing ###################

run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)


checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, random_seed, run_num_pretrained)
print("save checkpoint path : " + checkpoint_path)

#####################################################


############# print all hyper-parameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)

print("model saving frequency : " + str(save_model_freq) + " timesteps")
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("state space dimension : ", state_dim)
print("action space dimension : ", action_dim)

print("--------------------------------------------------------------------------------------------")

if has_continuous_action_space:
    print("Initializing a continuous action space policy")
    print("--------------------------------------------------------------------------------------------")
    print("starting std of action distribution : ", action_std)
    print("decay rate of std of action distribution : ", action_std_decay_rate)
    print("minimum std of action distribution : ", min_action_std)
    print("decay frequency of std of action distribution : " + str(action_std_decay_freq) + " timesteps")

else:
    print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps")
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0



# training loop
INTENTION_MAPPING = {'forward': 0, 'left': 1, 'right': 2}
with open('./testcases/milestone1_paths/' + args_.control_path) as f:
    lines = f.readlines()
    intention_li = []
    tile_pos_li = []
    for line in lines:
        cur_tile, high_level_action = line.split('), ')
        cur_tile_index = [int(cur_tile[1:].split(', ')[0]), int(cur_tile[1:].split(', ')[1])]
        intention = high_level_action[:-1]
        intention = torch.tensor(INTENTION_MAPPING[intention])
        intention_li.append(intention)
        tile_pos_li.append(cur_tile_index)

cur_intention_index = 0

while time_step <= max_training_timesteps:

    state = env.reset()
    current_ep_reward = 0

    for t in range(1, max_ep_len+1):

        # select action with policy
        # print('state original shape', state.shape)
        # print('state transformed shape', state_transformed.shape)

        ###----> all inputs that are taken into consideration


        ## 持续这一intention直到不在此行所在tile index
        action = ppo_agent.select_action(state, intention_li[cur_intention_index])

        state, reward, done, info = env.step(action)


        next_intention_index = next_pos_index = cur_intention_index + 1
        if next_pos_index < len(tile_pos_li):
            next_pos = tile_pos_li[next_pos_index]
        if info['curr_pos'] == next_pos: # move to next index
            cur_intention_index += 1

        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)

        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # if continuous action space; then decay action std of ouput action distribution
        if has_continuous_action_space and time_step % action_std_decay_freq == 0:
            ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)

            log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
            log_f.flush()

            log_running_reward = 0
            log_running_episodes = 0

        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / (print_running_episodes + 1)
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))

            print_running_reward = 0
            print_running_episodes = 0

        # save model weights
        if time_step % save_model_freq == 0:
            print("--------------------------------------------------------------------------------------------")
            print("saving model at : " + checkpoint_path)
            ppo_agent.save(checkpoint_path)
            print("model saved")
            print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
            print("--------------------------------------------------------------------------------------------")

        # break; if the episode is over
        if done:
            break

    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()
env.close()

# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")

training environment name : DuckietownEnv
current logging run number for DuckietownEnv :  62
logging at : PPO_logs/DuckietownEnv//PPO_DuckietownEnv_log_62.csv
save checkpoint path : PPO_preTrained/DuckietownEnv/PPO_DuckietownEnv_4_0.pth
--------------------------------------------------------------------------------------------
max training timesteps :  5000
max timesteps per episode :  1500
model saving frequency : 100 timesteps
log frequency : 3000 timesteps
printing average reward over episodes in last : 4 timesteps
--------------------------------------------------------------------------------------------
state space dimension :  307200
action space dimension :  2
--------------------------------------------------------------------------------------------
Initializing a continuous action space policy
--------------------------------------------------------------------------------------------
starting std of action distribution :  0.1
decay rate of std of action distribution :  0
m

/var/folders/zh/yzm7x5vn60912p940qstzmww0000gn/T/ipykernel_6320/531460410.py:127: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).permute(2, 0, 1)


Episode : 0 		 Timestep : 8 		 Average Reward : 0.0
Episode : 0 		 Timestep : 12 		 Average Reward : 0.0
Episode : 0 		 Timestep : 16 		 Average Reward : 0.0
Episode : 1 		 Timestep : 20 		 Average Reward : -503.72
Episode : 1 		 Timestep : 24 		 Average Reward : 0.0
Episode : 1 		 Timestep : 28 		 Average Reward : 0.0
Episode : 1 		 Timestep : 32 		 Average Reward : 0.0
Episode : 1 		 Timestep : 36 		 Average Reward : 0.0
Episode : 1 		 Timestep : 40 		 Average Reward : 0.0
Episode : 1 		 Timestep : 44 		 Average Reward : 0.0
Episode : 1 		 Timestep : 48 		 Average Reward : 0.0
Episode : 1 		 Timestep : 52 		 Average Reward : 0.0
Episode : 1 		 Timestep : 56 		 Average Reward : 0.0
Episode : 1 		 Timestep : 60 		 Average Reward : 0.0
Episode : 1 		 Timestep : 64 		 Average Reward : 0.0
Episode : 1 		 Timestep : 68 		 Average Reward : 0.0
Episode : 1 		 Timestep : 72 		 Average Reward : 0.0
Episode : 2 		 Timestep : 76 		 Average Reward : -598.41
Episode : 2 		 Timestep : 80 		 Average

> # **Part - III**

*   load and test preTrained networks on environments

In [106]:


print("============================================================================================")


#################################### Testing ###################################

env_name = "DuckietownEnv"
has_continuous_action_space = True
max_ep_len = 1500           # max timesteps in one episode
action_std = 0.1            # set same std for action distribution which was used while saving

total_test_episodes = 1    # total num of testing episodes

K_epochs = 80               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003           # learning rate for actor
lr_critic = 0.001           # learning rate for critic

#####################################################

class Args_:
    def __init__(self):
        # self.map_name = "map4_0"
        # self.seed = 2
        # self.start_tile = "1,13"
        # self.goal_tile = "3,3"
        # self.control_path = 'map4_0_seed2_start_1,13_goal_3,3.txt'

        # add intention
        self.map_name = "map4_0"
        self.seed = 4
        self.start_tile = "3,3"
        self.goal_tile = "10,4"
        self.control_path = 'map4_0_seed4_start_3,3_goal_10,4.txt'

args_ = Args_()
# simulator instantiation
env = DuckietownEnv(
    domain_rand=False,
    max_steps=1500,
    map_name=args_.map_name,
    seed=args_.seed,
    user_tile_start=args_.start_tile,
    goal_tile=args_.goal_tile,
    randomize_maps_on_reset=False
)

# state space dimension
state_dim = env.observation_space.shape[0]

# action space dimension
if has_continuous_action_space:
    action_dim = env.action_space.shape[0]
else:
    action_dim = env.action_space.n


# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std)


# preTrained weights directory

random_seed = args_.seed    #### set this to load a particular checkpoint trained on random seed
run_num_pretrained = 0      #### set this to load a particular checkpoint num


directory = "PPO_preTrained" + '/' + env_name + '/'
checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, random_seed, run_num_pretrained)
print("loading network from : " + checkpoint_path)

ppo_agent.load(checkpoint_path)

print("--------------------------------------------------------------------------------------------")


state = env.reset()
episode_reward = 0
actions = []
for t in range(1, max_ep_len+1):
    action = ppo_agent.select_action(state)
    actions.append(action)
    state, reward, done, _ = env.step(action)
    episode_reward += reward
    if done:
        break

map_img, goal, start_pos = env.get_task_info()
np.savetxt(args_.control_path, actions, delimiter=',')

# clear buffer
ppo_agent.buffer.clear()
env.close()


print("============================================================================================")

print('Reward: {}'.format(round(episode_reward, 2)))

print("============================================================================================")

loading network from : PPO_preTrained/DuckietownEnv/PPO_DuckietownEnv_4_0.pth
--------------------------------------------------------------------------------------------


TypeError: select_action() missing 1 required positional argument: 'intention'